In [1]:
# importando pacotes utilizados

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests as re
import time
import plotly.express as px

### Extração de dados a partir da API da NBA

In [2]:
#estrutura inicial dos dados
url_nba = f'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season=2021-22&SeasonType=Regular%20Season&StatCategory=PTS' # url da api com f-string para iterar listas
dados = re.get(url=url_nba).json() # fazendo request do json

colunas_iniciais = ['YEAR', 'SEASON'] + dados['resultSet']['headers'] # estrutura inicial do dataframe

df_final = pd.DataFrame(columns=colunas_iniciais) # iniciando o dataframe

# parametros api
tipo_season = ['Regular%20Season', 'Playoffs'] # tipo de temporada
year = ['2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21','2021-22', '2022-23'] # ano da temporada

for y in year: # iterando lista de anos de temporada
    for t in tipo_season: # iterando tipos de temporada playoff ou regular
        url_nba = f'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season={y}&SeasonType={t}&StatCategory=PTS' # url da api com f-string para iterar listas
        dados = re.get(url=url_nba).json() # fazendo request do json
        df1 = pd.DataFrame(dados['resultSet']['rowSet'], columns=dados['resultSet']['headers']) # request da informacao selecionada na api anteriormente
        df2 = pd.DataFrame({'YEAR':[y for i in range(len(df1))], # aplicando o ano e a temporada para cada linha 
                            'SEASON':[t for i in range(len(df1))]})
        df3 = pd.concat([df2, df1], axis=1) # concatenando em um dataframe ajustado
        df_final = pd.concat([df_final, df3], axis = 0) # empilhando as bases
        print(f'Dados obtidos: ano - {y}, temporada - {t}')
        #time.sleep(np.random.uniform(low=5, high = 10))
        
print ('Executado')


Dados obtidos: ano - 2010-11, temporada - Regular%20Season
Dados obtidos: ano - 2010-11, temporada - Playoffs
Dados obtidos: ano - 2011-12, temporada - Regular%20Season
Dados obtidos: ano - 2011-12, temporada - Playoffs
Dados obtidos: ano - 2012-13, temporada - Regular%20Season
Dados obtidos: ano - 2012-13, temporada - Playoffs
Dados obtidos: ano - 2013-14, temporada - Regular%20Season
Dados obtidos: ano - 2013-14, temporada - Playoffs
Dados obtidos: ano - 2014-15, temporada - Regular%20Season
Dados obtidos: ano - 2014-15, temporada - Playoffs
Dados obtidos: ano - 2015-16, temporada - Regular%20Season
Dados obtidos: ano - 2015-16, temporada - Playoffs
Dados obtidos: ano - 2016-17, temporada - Regular%20Season
Dados obtidos: ano - 2016-17, temporada - Playoffs
Dados obtidos: ano - 2017-18, temporada - Regular%20Season
Dados obtidos: ano - 2017-18, temporada - Playoffs
Dados obtidos: ano - 2018-19, temporada - Regular%20Season
Dados obtidos: ano - 2018-19, temporada - Playoffs
Dados obti

## Glossary
* GP - Games Played
* MIN - Minutes Played
* PTS - Points
* FGM - Field Goals Made
* FGA - Field Goals Attempted
* FG% - Field Goal Percentage
* 3PM - 3 Point Field Goals Made
* 3PA - 3 Point Field Goals Attempted
* 3P% - 3 Point Field Goal Percentage
* FTM - Free Throws Made
* FTA - Free Throws Attempted
* FT% - Free Throw Percentage
* OREB - Offensive Rebounds
* DREB - Defensive Rebounds
* REB - Rebounds
* AST - Assists
* STL - Steals
* BLK - Blocks
* TOV - Turnovers

In [5]:
# visualizando os dados para ajustes

df_final.head()

,YEAR,SEASON,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,2010-11,Regular%20Season,201142,1,Kevin Durant,1610612760,OKC,78,3038,711,...,533,214,88,76,218,159,2161,1946,0.98,0.40
1,2010-11,Regular%20Season,2544,2,LeBron James,1610612748,MIA,79,3063,758,...,590,554,124,50,284,163,2111,2258,1.95,0.44
2,2010-11,Regular%20Season,977,3,Kobe Bryant,1610612747,LAL,82,2779,740,...,419,388,99,12,243,172,2078,1754,1.60,0.41
3,2010-11,Regular%20Season,201565,4,Derrick Rose,1610612741,CHI,81,3026,711,...,330,623,85,51,278,136,2026,1872,2.24,0.31
4,2010-11,Regular%20Season,2405,5,Amar'e Stoudemire,1610612752,NYK,78,2870,744,...,636,202,71,150,250,274,1971,1918,0.81,0.28


In [6]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9374 entries, 0 to 216
Data columns (total 30 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   YEAR       9374 non-null   object 
 1   SEASON     9374 non-null   object 
 2   PLAYER_ID  9374 non-null   object 
 3   RANK       9374 non-null   object 
 4   PLAYER     9374 non-null   object 
 5   TEAM_ID    9374 non-null   object 
 6   TEAM       9374 non-null   object 
 7   GP         9374 non-null   object 
 8   MIN        9374 non-null   object 
 9   FGM        9374 non-null   object 
 10  FGA        9374 non-null   object 
 11  FG_PCT     9374 non-null   float64
 12  FG3M       9374 non-null   object 
 13  FG3A       9374 non-null   object 
 14  FG3_PCT    9374 non-null   float64
 15  FTM        9374 non-null   object 
 16  FTA        9374 non-null   object 
 17  FT_PCT     9374 non-null   float64
 18  OREB       9374 non-null   object 
 19  DREB       9374 non-null   object 
 20  REB        937